In [177]:
class Matrix:
    pass
class Vector:
    pass

class Matrix:
    def __init__(self, data):
        self.data = data
        self.m = len(data)          #rows
        self.n = len(data[0])       #cols

    def shape(self):
        return self.m, self.n
    
    def T(self):
        return Matrix([[self.data[j][i] for j in range(self.m)] for i in range(self.n)])

    def __str__(self):
        return '\n'.join(["|| " + (" ".join(map(str, row))) + " ||" for row in self.data])

    def __repr__(self):
        return f"Matrix({self.m}x{self.n})"

    def __add__(self, other : Matrix) -> Matrix:
        if isinstance(other, Matrix) and self.shape() == other.shape():
            return Matrix([[self.data[i][j] + other.data[i][j] for j in range(self.n)] for i in range(self.m)])
        else:
            raise ValueError("matrix add/sub error")
        
    def __mul__(self, other : Matrix | float) -> Matrix:
        if isinstance(other, Matrix):
            if self.n != other.m:
                raise ValueError("bad dimensions")
            return Matrix([[sum(self.data[i][k] * other.data[k][j] for k in range(self.n)) 
                      for j in range(other.n)] for i in range(self.m)])
        elif isinstance(other, (int, float)):
            return Matrix([[self.data[i][j] * other for j in range(self.n)] for i in range(self.m)])
        else:
            raise ValueError("unsupported operand for matrix multiplication")
        
    def __rmul__(self, other : Matrix | float) -> Matrix:
        return self * other
        
    def __sub__(self, other : Matrix) -> Matrix:
        return self + other * (-1)
    
    def __eq__(self, other : Matrix) -> bool:
        if not isinstance(other, Matrix):
            return False
        return self.data == other.data
    
    def is_square(self) -> bool:
        return self.m == self.n
    
    def minor(self, row, col) -> Matrix:
        return Matrix([self.data[i][:col] + self.data[i][col + 1:] for i in range(self.m) if i != row])
    
    def det(self) -> float:
        if not self.is_square():
            raise ValueError("attempt to get determinant of not square matrix")
        if self.m == 1:
            return self.data[0][0]
        if self.m == 2:
            return self.data[0][0] * self.data[1][1] - self.data[0][1] * self.data[1][0]
        det = 0
        for c in range(self.m):
            det += ((-1) ** c) * self.data[0][c] * self.minor(0, c).det()
        return det

    def tr(self) -> float:
        if not self.is_square():
            raise ValueError("attempt to get trace of not square matrix")
        trace = 0
        for i in range(self.n):
            trace += self.data[i][i]
        return trace
    
    def copy(self) -> Matrix:
        if isinstance(self, Vector):
            return Vector([item[0] for item in self.data])
        return Matrix([[item for item in row] for row in self.data])
    
    def __getitem__(self, index : int | tuple[int, int]) -> float:
        if isinstance(index, tuple) and len(index) == 2:
            row, col = index
            return self.data[row][col]
        elif isinstance(index, int):
            return self.data[index]
        else:
            raise IndexError("invalid matrix index")
    
    def __setitem__(self, index : int | tuple[int, int], value : float) -> None:
        if isinstance(index, tuple) and len(index) == 2:
            row, col = index
            self.data[row][col] = value
        elif isinstance(index, int):
            if len(value) != self.n:
                raise ValueError("invalid row size")
            self.data[index] = value
        else:
            raise IndexError("invalid matrix index")
    
def det(A : Matrix):
    return A.det()

def tr(A : Matrix):
    return A.tr()

class Vector(Matrix):
    def __init__(self, data):
        super().__init__([[item] for item in data])
    
    def __getitem__(self, index : int) -> float:
        return super().__getitem__((index, 0))
    
    def __setitem__(self, index : int, value : float) -> None:
        super().__setitem__((index, 0), value)

    def zero(size : int):
        return Vector([0 for i in range(size)])

def get_n1(A : Matrix):
    result = 0
    for i in range(A.m):
        for j in range(A.n):
            result += abs(A[i, j])
    return result

## Matrix and other settings

In [178]:
A = Matrix([[2, 3, -1], [4, 1, 2], [-3, 2, 1]])
f = Vector([9, 4, 0])
eps = 1e-15 #residual for iterational methods

print(det(A))

-47


## Jacobi

In [179]:
def find_solution_by_Jacobi(A : Matrix, f : Vector, x0 : Vector) -> float:
    n = A.shape()[0]
    x = x0.copy()
    x_dual = Vector.zero(n)
    while get_n1(A * x - f) > eps:
        for i in range(n):
            x_dual[i] = (-sum([A[i, j] * x[j] for j in range(n) if i != j]) + f[i]) / A[i, i]
        x, x_dual = x_dual, x
    return x

print(find_solution_by_Jacobi(A, f, Vector.zero(3)))

|| inf ||
|| -inf ||
|| inf ||


## Seidel

In [180]:
def find_solution_by_Seidel(A : Matrix, f : Vector, x0 : Vector) -> float:
    f = Vector([item[0] for item in (A.T() * f).data])
    A = A.T() * A
    n = A.shape()[0]
    x = x0.copy()
    x_dual = Vector.zero(n)
    while get_n1(A * x - f) > eps:
        for i in range(n):
            x_dual[i] = -(sum([A[i, j] * x_dual[j] for j in range(i) if i != j]) 
                          + sum([A[i, j] * x[j] for j in range(i+1, n) if i != j]) 
                          - f[i]) / A[i, i]
        x, x_dual = x_dual, x
    return x

x = find_solution_by_Seidel(A, f, Vector([0, 0, 0]))
print(x, "\n---\n", A*x, sep="")

|| 1.0 ||
|| 2.0 ||
|| -1.0 ||
---
|| 9.0 ||
|| 4.0 ||
|| 0.0 ||
